In [333]:
"""Using the movies dataset to model the content based recomender system
Author: Siva E
Version:1.0"""

'Using the movies dataset to model the content based recomender system\nAuthor: Siva E\nVersion:1.0'

In [334]:
"""Details of data are given below:
http://www.movielens.org/

contains three files as below

RATINGS FILE DESCRIPTION
UserID::MovieID::Rating::Timestamp

USERS FILE DESCRIPTION
UserID::Gender::Age::Occupation::Zip-code

MOVIES FILE DESCRIPTION
MovieID::Title::Genres"""

'Details of data are given below:\nhttp://www.movielens.org/\n\ncontains three files as below\n\nRATINGS FILE DESCRIPTION\nUserID::MovieID::Rating::Timestamp\n\nUSERS FILE DESCRIPTION\nUserID::Gender::Age::Occupation::Zip-code\n\nMOVIES FILE DESCRIPTION\nMovieID::Title::Genres'

In [335]:
# load required modules on demand and read in data to required format
import pandas as pd

In [336]:
# loading movies data
movies=pd.read_csv("/movies.dat",sep="::", encoding="latin-1", names=["MovieID", "MovieName", "Genre"],header=None)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [337]:
movies.head

<bound method NDFrame.head of       MovieID                           MovieName  \
0           1                    Toy Story (1995)   
1           2                      Jumanji (1995)   
2           3             Grumpier Old Men (1995)   
3           4            Waiting to Exhale (1995)   
4           5  Father of the Bride Part II (1995)   
...       ...                                 ...   
3878     3948             Meet the Parents (2000)   
3879     3949          Requiem for a Dream (2000)   
3880     3950                    Tigerland (2000)   
3881     3951             Two Family House (2000)   
3882     3952               Contender, The (2000)   

                             Genre  
0      Animation|Children's|Comedy  
1     Adventure|Children's|Fantasy  
2                   Comedy|Romance  
3                     Comedy|Drama  
4                           Comedy  
...                            ...  
3878                        Comedy  
3879                         Drama  


In [338]:
# loading ratings data
ratings=pd.read_csv("/ratings.dat",sep="::", encoding="latin-1", names=["UserID","MovieID","Rating","Timestamp"],header=None)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [339]:
#  To use countvectoriser to vectorise Genre column
from sklearn.feature_extraction.text import CountVectorizer
tfidf=CountVectorizer(analyzer="word",stop_words="english")

In [340]:
# correcting the special charcters in genres
movies["Genre"]=movies["Genre"].map(lambda Genre: Genre.replace("|"," "))

In [341]:
movies["Genre"]=movies["Genre"].map(lambda Genre: Genre.replace("-",""))

In [342]:
genres=tfidf.fit_transform(movies["Genre"])

In [343]:
genre_names=list(tfidf.get_feature_names_out())

In [344]:
# converting spare matrix output to dense pandas df
genre_df=pd.DataFrame.sparse.from_spmatrix(genres,columns=genre_names)

In [345]:
# combining movies and vectorised Genre df
movie_genre=pd.concat([movies,genre_df],axis=1)

In [346]:
movie_genre.drop(["MovieName","Genre"],axis=1,inplace=True)

In [347]:
movie_genre.set_index("MovieID",inplace=True)

In [348]:
# forming a column for mean ratings foreach movie
ratings_grouped=ratings.groupby("MovieID")["Rating"].mean()

In [349]:
# merging movie, genre, and rating data
movies_ratings=pd.merge(movie_genre,ratings_grouped,on="MovieID")

In [350]:
# finding similarity of the vectors using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [351]:
similarity_mat=cosine_similarity(movies_ratings.to_numpy())

In [352]:
# reference to lookup movieID from index
MID=movies_ratings.index.to_list()

In [353]:
# input a mobiename from data
input_movie=input("enter movie: ")


enter movie: Wanted: Dead or Alive (1987)


In [354]:
# Get movieID of the given movie
movie_ID=movies.at[movies[movies["MovieName"]==input_movie].index[0],"MovieID"]

In [355]:
# get top 5 similar movies from similarity matrix corresponding to given movie
top5_list_tuple=sorted(list(enumerate(similarity_mat[MID.index(movie_ID)])), key=lambda x:x[1],reverse=True)
ID_list=[]
for tups in top5_list_tuple[:6]:
  ID_list.append(tups[0])

In [356]:
# output the 5 simolar movie names to given movie
for Ids in ID_list[1:]:
  print(movies[movies["MovieID"]==Ids]["MovieName"].values[0])

Born to Win (1971)
Never Talk to Strangers (1995)
Man Who Knew Too Much, The (1956)
Somewhere in Time (1980)
Gumby: The Movie (1995)
